In [8]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score


In [37]:
def process_csv_feautures(file_paths):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instruction']
            cyc = row['cpu-cycle']
            ipc = inst/cyc
            features.append([row['instruction'],row['cpu-cycle'], row['armv8_pmuv3/stall_backen'],row['armv8_pmuv3/stall_fronten'],row['armv8_pmuv3/br_mis_pre'],row['armv8_pmuv3/br_pre'], row['armv8_pmuv3/br_retire'], row['armv8_pmuv3/inst_spe'],row['armv8_pmuv3/inst_retire'],row['armv8_pmuv3/l1d_cach'],row['armv8_pmuv3/l1d_cache_refil'],row['armv8_pmuv3/l1d_cache_w'],row['armv8_pmuv3/l1d_tl'],row['armv8_pmuv3/l1d_tlb_refil'],row['armv8_pmuv3/l1i_cach'],row['armv8_pmuv3/l1i_cache_refil'],row['armv8_pmuv3/l1i_tl'],row['armv8_pmuv3/l1i_tlb_refil'], row['armv8_pmuv3/l2d_cache_allocat'],row['armv8_pmuv3/l2d_cache_refil'],row['armv8_pmuv3/l2d_cache_w'],row['armv8_pmuv3/l2d_tl'],row['armv8_pmuv3/l2d_tlb_refil'],row['armv8_pmuv3/l3d_cach'],row['armv8_pmuv3/l3d_cache_allocat'],row['armv8_pmuv3/l3d_cache_refil'],row['armv8_pmuv3/mem_acces'],ipc])
            #, ipc])
                
                
                #[row['armv8_pmuv3/stall_backen'], row['armv8_pmuv3/br_mis_pre'], row['armv8_pmuv3/l2d_cache_refil'], row['armv8_pmuv3/l1i_cache_refil'], row['armv8_pmuv3/br_retire'], row['armv8_pmuv3/inst_retire'], row['armv8_pmuv3/l1d_cache_refil'], ipc])
            
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instruction'] / row['cpu-cycle'])  
    return all_ipc_values

in_order_files = [
    'inst_aligned_InO_500.csv',
    'inst_aligned_InO_502.csv',
    'inst_aligned_InO_505.csv',
    'inst_aligned_InO_520.csv',
    'inst_aligned_InO_523.csv',
    'inst_aligned_InO_525.csv',
    'inst_aligned_InO_531.csv',
    'inst_aligned_InO_541.csv',
    'inst_aligned_InO_548.csv',
    'inst_aligned_InO_557.csv',
    'inst_aligned_InO_503.csv',
    'inst_aligned_InO_507.csv',
    'inst_aligned_InO_508.csv',
    'inst_aligned_InO_510.csv',
    'inst_aligned_InO_511.csv',
    'inst_aligned_InO_519.csv',
    'inst_aligned_InO_521.csv',
    'inst_aligned_InO_526.csv',
    'inst_aligned_InO_527.csv',
    'inst_aligned_InO_538.csv',
    'inst_aligned_InO_544.csv',
    'inst_aligned_InO_549.csv',
    'inst_aligned_InO_554.csv'
]

out_of_order_files = [
    'inst_aligned_OOO_500.csv',
    'inst_aligned_OOO_502.csv',
    'inst_aligned_OOO_505.csv',
    'inst_aligned_OOO_520.csv',
    'inst_aligned_OOO_523.csv',
    'inst_aligned_OOO_525.csv',
    'inst_aligned_OOO_531.csv',
    'inst_aligned_OOO_541.csv',
    'inst_aligned_OOO_548.csv',
    'inst_aligned_OOO_557.csv',
    'inst_aligned_OOO_503.csv',
    'inst_aligned_OOO_507.csv',
    'inst_aligned_OOO_508.csv',
    'inst_aligned_OOO_510.csv',
    'inst_aligned_OOO_511.csv',
    'inst_aligned_OOO_519.csv',
    'inst_aligned_OOO_521.csv',
    'inst_aligned_OOO_526.csv',
    'inst_aligned_OOO_527.csv',
    'inst_aligned_OOO_538.csv',
    'inst_aligned_OOO_544.csv',
    'inst_aligned_OOO_549.csv',
    'inst_aligned_OOO_554.csv'
]    
    
in_order_hardware_counters_features = process_csv_feautures(in_order_files)
out_of_order_ipc = process_csv_ipc(out_of_order_files)


# 
data = {
    'In Order': in_order_hardware_counters_features,
    'Out of Order': out_of_order_ipc
}

length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))

# Shorten the longen array
in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
out_of_order_ipc = out_of_order_ipc[:length]

# separate features and target
X = pd.DataFrame(in_order_hardware_counters_features, columns=['instruction', 'cpu-cycle', 'armv8_pmuv3/stall_backen', 'armv8_pmuv3/stall_fronten', 'armv8_pmuv3/br_mis_pre', 'armv8_pmuv3/br_pre', 'armv8_pmuv3/br_retire', 'armv8_pmuv3/inst_spe','armv8_pmuv3/inst_retire','armv8_pmuv3/l1d_cach','armv8_pmuv3/l1d_cache_refil', 'armv8_pmuv3/l1d_cache_w','armv8_pmuv3/l1d_tl','armv8_pmuv3/l1d_tlb_refil','arm8_pmuv3/l1i_cach','armv8_pmuv3/l1i_cache_refil','armv8_pmuv3/l1i_tl','armv8_pmuv3/l1i_tlb_refil','armv8_pmuv3/l2d_cache_allocat','armv8_pmuv3/l2d_cache_refil','armv8_pmuv3/l2d_cache_w','armv8_pmuv3/l2d_tl','armv8_pmuv3/l2d_tlb_refil','armv8_pmuv3/l3d_cach','armv8_pmuv3/l3d_cache_allocat','armv8_pmuv3/l3d_cache_refil','armv8_pmuv3/mem_acces','ipc'])  # Features
y = pd.Series(out_of_order_ipc) 

# split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100)
gb_model = GradientBoostingRegressor(n_estimators=100)
ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
ensemble_model.fit(X_train, y_train)
ensemble_predictions = ensemble_model.predict(X_test)
# Calculate R²
ensemble_r2 = r2_score(y_test, ensemble_predictions)


#bruh wut
train_predictions = ensemble_model.predict(X_train)
test_predictions = ensemble_model.predict(X_test)

# Calculate R² for training and testing sets
train_r2 = r2_score(y_train, train_predictions)
test_r2 = r2_score(y_test, test_predictions)

print(f"Training R²: {train_r2}")
print(f"Testing R²: {test_r2}")




# Create a DataFrame for Actual and Predicted values
results_df = pd.DataFrame({
    'In Order IPC': X_test['ipc'],
    'Out of Order IPC (Actual)': y_test,
    'Out of Order IPC (Predicted)': ensemble_predictions
})

results_df.to_csv('predicted_vs_actual_aligned.csv', index=False)
print("Predicted vs Actual values have been written to 'predicted_vs_actual_aligned.csv'.")


Training R²: 0.9905922749283388
Testing R²: 0.9666499299301441
Predicted vs Actual values have been written to 'predicted_vs_actual_aligned.csv'.
